In [1]:
import os
import numpy as np
import subprocess as sp
import pickle as pkl
import pandas as pd
import shutil

In [2]:
mod_dir = os.getcwd()

In [3]:
hgs_mod_dir = os.path.join(mod_dir,'hgs')
model_name = 'lys'
grok_file_path = os.path.join(hgs_mod_dir,model_name + '_e.grok')

cmod_dir = os.path.join(mod_dir,'coupled')
cmod_hgs_dir = os.path.join(cmod_dir,'hgs')
cmod_dssat_dir = os.path.join(cmod_dir,'dssat')
cmapping_dir = os.path.join(cmod_dir,'mapping')

In [4]:
grok_file_stem = model_name + '_e'

In [5]:
e_mapping_pkl_path = os.path.join(cmapping_dir,'lys_elem_mapping.p')

In [6]:
id = 'test'
top_node_sheet_number = 15
mdl = 1

In [7]:
def BuildDailyFluxFiles(grok_file_stem,coupled_mod_hgs_dir,coupled_mod_dssat_dir,i,mdl):
    # Get path to file
    nfmb_path = os.path.join(coupled_mod_hgs_dir,grok_file_stem + 'o.nodal_fluid_mass_balance.'+id+'.dat')
    # Open file and grab area and variable list
    with open(nfmb_path,'r') as file:
        for i,line in enumerate(file.readlines()):
            if i == 1:
                area = float(line.split(':')[1].strip())
            elif i == 3:
                vars = [x.replace('"','') for x in line.strip().split('=')[1].split(',')]
            else:
                if 'Zone T=' in line:
                    start_line = i+1
    # Load whole table as df
    df = pd.read_csv(nfmb_path, skiprows = start_line, names = vars, delim_whitespace=True)
    # Get column of time increment
    df['Time Inc'] = df['Time'].diff()
    df.loc[0,'Time Inc'] = df['Time'].values[0]
    # Iterate thru node sheets to get drainage fluxes
    Net_Q_List = []
    for i in np.flip(np.arange(top_node_sheet_number-11,top_node_sheet_number)):
        net_q_name = 'Net_Q_{:02d}'.format(i)
        df[net_q_name] = df.apply(lambda x: (x['QVD+{:02d}'.format(i)]+-1*(x['QVD-{:02d}'.format(i)]))*x['Time Inc']/area*100., axis = 1)
        Net_Q_List.append(net_q_name)
    # Iterate through days - DON'T NEED ITERATION
    for day in np.arange(0,275):
        Net_Q_Vals = []
        sub_df = df.loc[(df['Time'] > day) & (df['Time'] <= day+1), Net_Q_List]
        for name in Net_Q_List:
            Net_Q_Vals.append(sub_df[name].sum())
        # Write out to file
        drn_path = os.path.join(coupled_mod_dssat_dir,'{}_{}_DRN.inp'.format(mdl,day))
        with open(drn_path,'w') as file:
            line = ''
            for val in Net_Q_Vals:
                line += (f"{val:7.4f}"+'   ')
            for i in range(9):
                line +=(f" 0.0000"+'   ')
            file.write(line)
    # Resample df to get daily sum
    r_df = df.loc[:,['Time'] + [col for col in df.columns if 'Net_Q_' in col]]
    r_df["Time"] = pd.to_timedelta(r_df["Time"], unit="D")
    df_resampled = r_df.set_index("Time").resample("D").sum().reset_index()
    return df_resampled


In [8]:
# Create Flux Files
df = BuildDailyFluxFiles(grok_file_stem,hgs_mod_dir,cmod_dssat_dir,id,mdl)

In [9]:
df.to_excel('test2.xlsx')